In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
"""
COMPLETE XGBOOST PIPELINE - MINIMAL VERSION
Run this as a single Kaggle notebook cell by cell

Dataset: ASINS Voice Database
Goal: Train ultra-fast XGBoost model for AI vs Human voice detection
Expected: 96-98% accuracy, 5-10ms inference
"""

'\nCOMPLETE XGBOOST PIPELINE - MINIMAL VERSION\nRun this as a single Kaggle notebook cell by cell\n\nDataset: ASINS Voice Database\nGoal: Train ultra-fast XGBoost model for AI vs Human voice detection\nExpected: 96-98% accuracy, 5-10ms inference\n'

In [2]:
# ============================================================================
# CELL 1: Install Dependencies
# ============================================================================
!pip install xgboost lightgbm -q

In [3]:
# ============================================================================
# CELL 2: Imports
# ============================================================================
import os
import numpy as np
import pandas as pd
import librosa
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import gc
import pickle
import time

print("✓ All imports successful")

✓ All imports successful


In [17]:
# ============================================================================
# CELL 3: Configuration
# ============================================================================
# Dataset paths (adjust if needed)
BASE_PATH = '/kaggle/input/asins-voice/build'
REAL_CLIPS_PATH = f'{BASE_PATH}/clips'
AI_CLIPS_PATH = f'{BASE_PATH}/clips_AI'

# Feature extraction parameters
SAMPLE_RATE = 16000
N_LFCC = 40
N_FFT = 2048
HOP_LENGTH = 512
MAX_DURATION = 10

# Dataset size
DATASET_SIZE = 136000  # 5K real + 5K AI
TEST_SIZE = 0.2
RANDOM_STATE = 42

print("✓ Configuration set")
print(f"Dataset size: {DATASET_SIZE} samples")
print(f"Feature extraction: {N_LFCC} LFCC, {SAMPLE_RATE}Hz, {MAX_DURATION}s")

✓ Configuration set
Dataset size: 136000 samples
Feature extraction: 40 LFCC, 16000Hz, 10s


In [18]:
# ============================================================================
# CELL 4: Dataset Preparation
# ============================================================================
print("\n" + "="*80)
print("STEP 1: DATASET PREPARATION")
print("="*80)

# Collect audio files
print("\n[1/5] Collecting audio files...")
real_files = [f for f in os.listdir(REAL_CLIPS_PATH) if f.endswith(('.mp3', '.wav', '.ogg', '.flac'))]
ai_files = [f for f in os.listdir(AI_CLIPS_PATH) if f.endswith(('.mp3', '.wav', '.ogg', '.flac'))]

print(f"Found {len(real_files)} real files")
print(f"Found {len(ai_files)} AI files")

# Find matched pairs
print("\n[2/5] Finding matched pairs...")
real_set = set(real_files)
ai_set = set(ai_files)
matched_files = list(real_set.intersection(ai_set))

print(f"Found {len(matched_files)} matched pairs")

# Sample balanced subset
print("\n[3/5] Sampling balanced subset...")
np.random.seed(RANDOM_STATE)
samples_per_class = DATASET_SIZE // 2

if len(matched_files) < samples_per_class:
    print(f"⚠ Warning: Only {len(matched_files)} pairs available, using all")
    samples_per_class = len(matched_files)

selected_files = np.random.choice(matched_files, size=samples_per_class, replace=False)

# Create dataframe
print("\n[4/5] Creating dataset...")
data = []
for filename in selected_files:
    data.append({
        'filename': filename,
        'filepath': os.path.join(REAL_CLIPS_PATH, filename),
        'label': 0,
        'label_text': 'REAL'
    })
    data.append({
        'filename': filename,
        'filepath': os.path.join(AI_CLIPS_PATH, filename),
        'label': 1,
        'label_text': 'AI_GENERATED'
    })

df = pd.DataFrame(data)
df = df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

print(f"Total samples: {len(df)}")
print(f"\nClass distribution:")
print(df['label_text'].value_counts())

# Train-test split
print("\n[5/5] Creating train-test split...")
train_df, test_df = train_test_split(
    df, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE,
    stratify=df['label']
)

print(f"Train: {len(train_df)} samples")
print(f"Test: {len(test_df)} samples")
print("✓ Dataset preparation complete!")



STEP 1: DATASET PREPARATION

[1/5] Collecting audio files...
Found 68030 real files
Found 68030 AI files

[2/5] Finding matched pairs...
Found 68030 matched pairs

[3/5] Sampling balanced subset...

[4/5] Creating dataset...
Total samples: 136000

Class distribution:
label_text
REAL            68000
AI_GENERATED    68000
Name: count, dtype: int64

[5/5] Creating train-test split...
Train: 108800 samples
Test: 27200 samples
✓ Dataset preparation complete!


In [19]:
# ============================================================================
# CELL 5: Feature Extraction Functions
# ============================================================================
print("\n" + "="*80)
print("STEP 2: FEATURE EXTRACTION")
print("="*80)

def extract_lfcc(audio_path):
    """Extract LFCC features from audio file."""
    try:
        y, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=MAX_DURATION)
        
        lfcc = librosa.feature.mfcc(
            y=y, 
            sr=sr, 
            n_mfcc=N_LFCC,
            n_fft=N_FFT,
            hop_length=HOP_LENGTH
        )
        
        target_length = int(MAX_DURATION * sr / HOP_LENGTH)
        
        if lfcc.shape[1] < target_length:
            pad_width = target_length - lfcc.shape[1]
            lfcc = np.pad(lfcc, ((0, 0), (0, pad_width)), mode='constant')
        else:
            lfcc = lfcc[:, :target_length]
        
        return lfcc
    
    except Exception as e:
        print(f"Error: {audio_path}: {e}")
        target_length = int(MAX_DURATION * SAMPLE_RATE / HOP_LENGTH)
        return np.zeros((N_LFCC, target_length))

def extract_statistical_features(lfcc_matrix):
    """
    Extract statistical features from LFCC.
    This is MUCH faster than using raw LFCC for XGBoost.
    
    Input: (n_lfcc, time_steps) e.g., (40, 313)
    Output: (n_features,) e.g., (365,)
    """
    feat = []
    
    # Per-coefficient statistics (40 coefficients × 9 stats = 360 features)
    feat.extend(np.mean(lfcc_matrix, axis=1))       # 40
    feat.extend(np.std(lfcc_matrix, axis=1))        # 40
    feat.extend(np.min(lfcc_matrix, axis=1))        # 40
    feat.extend(np.max(lfcc_matrix, axis=1))        # 40
    feat.extend(np.percentile(lfcc_matrix, 25, axis=1))  # 40
    feat.extend(np.percentile(lfcc_matrix, 50, axis=1))  # 40
    feat.extend(np.percentile(lfcc_matrix, 75, axis=1))  # 40
    feat.extend(np.max(lfcc_matrix, axis=1) - np.min(lfcc_matrix, axis=1))  # 40 (range)
    feat.extend(np.percentile(lfcc_matrix, 75, axis=1) - np.percentile(lfcc_matrix, 25, axis=1))  # 40 (IQR)
    
    # Overall statistics (5 features)
    feat.extend([
        np.mean(lfcc_matrix),
        np.std(lfcc_matrix),
        np.min(lfcc_matrix),
        np.max(lfcc_matrix),
        np.median(lfcc_matrix)
    ])
    
    return np.array(feat)

print("✓ Feature extraction functions defined")


STEP 2: FEATURE EXTRACTION
✓ Feature extraction functions defined


In [ ]:
# ============================================================================
# CELL 6: Extract Features for Training Set
# ============================================================================
print("\n[1/2] Extracting TRAINING features...")
print("This will take 10-20 minutes...")

X_train = []
y_train = []

for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Train"):
    # Extract LFCC
    lfcc = extract_lfcc(row['filepath'])
    
    # Extract statistical features
    features = extract_statistical_features(lfcc)
    
    X_train.append(features)
    y_train.append(row['label'])
    
    # Clear memory periodically
    if (idx + 1) % 1000 == 0:
        gc.collect()

X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"✓ Training features extracted: {X_train.shape}")
print(f"  Samples: {X_train.shape[0]}")
print(f"  Features per sample: {X_train.shape[1]}")

# Save checkpoint
np.savez_compressed('features_train_xgb.npz', X=X_train, y=y_train)
print("✓ Saved checkpoint: features_train_xgb.npz")



[1/2] Extracting TRAINING features...
This will take 10-20 minutes...


Train:  24%|██▍       | 26253/108800 [11:56<29:52, 46.04it/s]  

In [ ]:
# ============================================================================
# CELL 7: Extract Features for Test Set
# ============================================================================
print("\n[2/2] Extracting TEST features...")

X_test = []
y_test = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Test"):
    lfcc = extract_lfcc(row['filepath'])
    features = extract_statistical_features(lfcc)
    
    X_test.append(features)
    y_test.append(row['label'])
    
    if (idx + 1) % 1000 == 0:
        gc.collect()

X_test = np.array(X_test)
y_test = np.array(y_test)

print(f"✓ Test features extracted: {X_test.shape}")

# Save checkpoint
np.savez_compressed('features_test_xgb.npz', X=X_test, y=y_test)
print("✓ Saved checkpoint: features_test_xgb.npz")

print("\n✓ Feature extraction complete!")
print(f"  Train: {X_train.shape}")
print(f"  Test: {X_test.shape}")

In [ ]:
# ============================================================================
# CELL 8: Train XGBoost Model
# ============================================================================
import lightgbm as lgb
print("\n" + "="*80)
print("STEP 3: TRAIN XGBOOST MODEL")
print("="*80)

# XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'gamma': 0,
    'reg_alpha': 0.1,
    'reg_lambda': 1,
    'tree_method': 'hist',
    'random_state': 42,
    'n_jobs': -1,
    'eval_metric': 'logloss'
}

print("\nTraining XGBoost with parameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

# Train model
print("\nTraining...")
start_time = time.time()

model = xgb.XGBClassifier(**params)
model.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)

training_time = time.time() - start_time

print(f"\n✓ Training completed in {training_time:.2f} seconds ({training_time/60:.1f} minutes)")


In [ ]:
# ============================================================================
# CELL 9: Evaluate Model
# ============================================================================
print("\n" + "="*80)
print("STEP 4: EVALUATION")
print("="*80)

# Predictions
print("\nMaking predictions on test set...")
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n🎯 Accuracy: {accuracy*100:.2f}%")

# Classification report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred, 
                          target_names=['REAL', 'AI_GENERATED'],
                          digits=4))

# Confusion matrix
print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f"\nTrue Negatives (REAL → REAL): {cm[0][0]}")
print(f"False Positives (REAL → AI): {cm[0][1]}")
print(f"False Negatives (AI → REAL): {cm[1][0]}")
print(f"True Positives (AI → AI): {cm[1][1]}")

# Speed benchmark
print("\n⚡ Speed Benchmark:")
n_samples = 100
test_samples = X_test[:n_samples]

start_time = time.time()
for i in range(n_samples):
    _ = model.predict_proba(test_samples[i:i+1])
avg_inference_time = (time.time() - start_time) / n_samples * 1000

print(f"  Average inference time: {avg_inference_time:.2f} ms/sample")
print(f"  Throughput: {1000/avg_inference_time:.0f} samples/second")


In [ ]:
# ============================================================================
# CELL 10: Save Model
# ============================================================================
print("\n" + "="*80)
print("STEP 5: SAVE MODEL")
print("="*80)

# Save XGBoost model
model.save_model('model_xgboost_full.json')
model.save_model('model_xgboost_full.ubj')  # Fastest format

# Save as pickle for compatibility
with open('model_xgboost_full.pkl', 'wb') as f:
    pickle.dump(model, f)

# Calculate sizes
import os
json_size = os.path.getsize('model_xgboost.json') / (1024 * 1024)
pkl_size = os.path.getsize('model_xgboost.pkl') / (1024 * 1024)

print(f"✓ Models saved:")
print(f"  model_xgboost.json ({json_size:.2f} MB)")
print(f"  model_xgboost.ubj (fastest loading)")
print(f"  model_xgboost.pkl ({pkl_size:.2f} MB)")

In [ ]:
# ============================================================================
# CELL 11: Final Summary
# ============================================================================
print("\n" + "="*80)
print("COMPLETE! ✅")
print("="*80)

print(f"\n📊 FINAL RESULTS:")
print(f"  Accuracy:         {accuracy*100:.2f}%")
print(f"  Model size:       {json_size:.2f} MB")
print(f"  Inference speed:  {avg_inference_time:.2f} ms")
print(f"  Training time:    {training_time/60:.1f} minutes")

print(f"\n💾 SAVED FILES:")
print(f"  • model_xgboost.json (deploy this)")
print(f"  • model_xgboost.ubj (fastest)")
print(f"  • model_xgboost.pkl (Python)")
print(f"  • features_train_xgb.npz (checkpoint)")
print(f"  • features_test_xgb.npz (checkpoint)")

print(f"\n🚀 USAGE IN PRODUCTION:")
print("""
import xgboost as xgb
import numpy as np
import librosa

# Load model
model = xgb.Booster()
model.load_model('model_xgboost.json')

# Extract features from new audio
y, sr = librosa.load('audio.mp3', sr=16000, duration=10)
lfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
features = extract_statistical_features(lfcc)

# Predict
dmatrix = xgb.DMatrix(features.reshape(1, -1))
prediction = model.predict(dmatrix)[0]
classification = "AI_GENERATED" if prediction > 0.5 else "REAL"
confidence = prediction if prediction > 0.5 else 1 - prediction

print(f"Classification: {classification}")
print(f"Confidence: {confidence:.2f}")
""")

print("\n✅ XGBoost model ready for deployment!")
print("="*80)

In [ ]:
# ============================================================================
# CELL 12: Quick Test on Random Sample
# ============================================================================
print("\n" + "="*80)
print("BONUS: QUICK TEST")
print("="*80)

# Test on a random sample
random_idx = np.random.randint(0, len(test_df))
test_row = test_df.iloc[random_idx]

print(f"\nTesting on: {test_row['filename']}")
print(f"True label: {test_row['label_text']}")

# Extract features
lfcc = extract_lfcc(test_row['filepath'])
features = extract_statistical_features(lfcc).reshape(1, -1)

# Predict
start = time.time()
pred_proba = model.predict_proba(features)[0]
inference_time = (time.time() - start) * 1000

prediction = "AI_GENERATED" if pred_proba[1] > 0.5 else "REAL"
confidence = max(pred_proba)

print(f"\nPrediction: {prediction}")
print(f"Confidence: {confidence:.2%}")
print(f"Inference time: {inference_time:.2f} ms")
print(f"Result: {'✓ CORRECT' if prediction == test_row['label_text'] else '✗ WRONG'}")

print("\n" + "="*80)
print("ALL DONE! 🎉")
print("="*80)
